In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import sys
import os
from model.dcgan import Generator as GeneratorDCGAN
from model.predictor import PredictorDCGAN
from utils import modify_state_dict

ModuleNotFoundError: No module named 'pytorch_lightning'

In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_resume = False
gen_path = './generatory-surgery/dcgan_checkpoints/netG_final.pth'
model_folder = './trained_encoder/dcgan_results'
num_epochs = 1000
batch_size = 64

In [2]:
class DCGAN_Encoder(pl.LightningModule):
    def __init__(self):
      super().__init__()
      self.encoder = DCGAN_Encoder_Cut1(nc=3, ndf=64).to(DEVICE)
      if train_resume:
        print("Resuming training for the encoder")
        optimizer=lbfgs.n_steps=25.z_lr=0.1.restarts=3.n_cuts=1.z_init_mode=clamped_normal.limit=1
        enc_train_path = '%s/encoder_bs' % (model_folder)
        self.encoder.load_state_dict(torch.load(enc_train_path))

      # Load generator
      state_dict = torch.load(gen_path, map_location=DEVICE)
      params = state_dict['params']
      generator_state = modify_state_dict(state_dict['generator'])
      gen = dcgan_generator(params)
      gen.load_state_dict(generator_state)
      gen = gen.eval().to(DEVICE)
      self.gen = gen
    
    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
      return optimizer

    def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      x = x.view(x.size(0), -1)
      z = self.encoder(x)    
      x_hat = self.decoder(z)
      loss = F.mse_loss(x_hat, x)
      self.log('train_loss', loss)
      return loss

    def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      x = x.view(x.size(0), -1)
      z = self.encoder(x)
      x_hat = self.decoder(z)
      loss = F.mse_loss(x_hat, x)
      self.log('val_loss', loss)

SyntaxError: invalid syntax (<ipython-input-2-afaf0419f0c3>, line 7)

In [3]:
# data
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

train_loader = DataLoader(mnist_train, batch_size=32)
val_loader = DataLoader(mnist_val, batch_size=32)

# model
model = DCGAN_Encoder()

# training
trainer = pl.Trainer(gpus=4, num_nodes=8, precision=16, limit_train_batches=0.5)
trainer.fit(model, train_loader, val_loader)

9913344it [00:02, 4564826.78it/s]                             


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



29696it [00:00, 1241320.03it/s]          

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



1649664it [00:02, 702166.87it/s]                             

RuntimeError: File not found or corrupted.